In [4]:
import xarray as xr

def unit_convertor():
    sec_to_yr = 60 * 60 * 24 * 365
    mol_to_g = 12
    g_to_Pg = 1E-15
    convertor = sec_to_yr * mol_to_g * g_to_Pg
    return convertor

def get_npp_data(path, scenario, model_id, variant_id, grid, time, lat_label="lat", lon_label="lon", rolling_time= 12*5, built_in_area=False, area_var = 'areacello'):

    # generate path to netcdf file
    monthly_npp_prefix = "intpp_Omon"
    area_prefix = "areacello_Ofx"
    nc_ext = ".nc"

    if grid == None:
        area_prefix = "areacello_fx"
        nc_npp = '_'.join([monthly_npp_prefix, model_id, scenario, variant_id, time]) + nc_ext
        nc_area = '_'.join([area_prefix, model_id, scenario, "r0i0p0"]) + nc_ext
    else:
        nc_npp = '_'.join([monthly_npp_prefix, model_id, scenario, variant_id, grid, time]) + nc_ext
        nc_area = '_'.join([area_prefix, model_id, scenario, variant_id, grid]) + nc_ext

    full_path_npp = f"{path}/{scenario}/{nc_npp}"
    full_path_area = f"{path}/{scenario}/{nc_area}"

    # Read in data
    if "*" in full_path_npp:
        model = xr.open_mfdataset(full_path_npp)
    else:
        model = xr.open_dataset(full_path_npp)

    if built_in_area:
        area = model
    else:
        area = xr.open_dataset(full_path_area)

    npp = model['intpp'] * area[area_var] * unit_convertor()
    npp_rolling= npp.sum(dim=[lat_label,lon_label]).sel(time=slice("2022-01", "2100-01")).rolling(time=rolling_time, center=True).mean()

    if isinstance(npp_rolling.indexes['time'], xr.coding.cftimeindex.CFTimeIndex):
        time = npp_rolling.indexes['time'].to_datetimeindex()
    else:
        time = npp_rolling.time.values
    npp = npp_rolling.values
    # percentage to the first available value
    npp_percent = (npp/npp[30] - 1) * 100

    return time, npp_percent

In [5]:
import matplotlib.pyplot as plt
import matplotlib.transforms as mtransforms
import matplotlib as mpl

mpl.rcParams.update(mpl.rcParamsDefault)
mpl.rcParams["font.family"] = "Fira Sans"

fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(12, 5), tight_layout=True, sharey=True)
plt.set_cmap('tab20c')
trans = mtransforms.ScaledTranslation(10/72, -5/72, fig.dpi_scale_trans)

ax1.text(0.05, 0.9, "a", transform=ax1.transAxes + trans, weight="bold", fontsize=15)
ax2.text(0.05, 0.9, "b", transform=ax2.transAxes + trans, weight="bold", fontsize=15)

for ax in (ax1, ax2):
    ax.set_xlabel("Year")
    ax.grid(True, which='both', linestyle='--', linewidth=0.3)
    ax.legend()

    for position in ['top','bottom','left','right']:
        ax.spines[position].set_linewidth(1)
        ax.spines[position].set_color("black")

def plot_npp(ax, *args, **kwargs):
    time, npp = get_npp_data(*args)
    return (ax.plot(time, npp, **kwargs))

# Plot CMIP6, CESM2
plot_npp(ax1, "~/Downloads/cmip", "ssp126", "CESM2", "r4i1p1f1", "gn", "*", "nlat","nlon", linestyle='-', label="CESM2")
plot_npp(ax2, "~/Downloads/cmip", "ssp585", "CESM2", "r4i1p1f1", "gn", "*","nlat","nlon",  linestyle="--", label="CESM2")

# Plot CMIP6, CanESM5
plot_npp(ax1, "~/Downloads/cmip", "ssp126", "CanESM5", "r1i1p2f1", "gn", "201501-210012", "i","j", linestyle="-", label="CanESM5")
plot_npp(ax2,  "~/Downloads/cmip", "ssp585", "CanESM5", "r1i1p2f1", "gn", "201501-210012","i","j", linestyle="--", label="CanESM5")

# Plot CMIP6, MIROC
plot_npp(ax1, "~/Downloads/cmip", "ssp126", "MIROC-ES2L", "r1i1p1f2", "gn", "201501-210012", "x","y", linestyle="-", label="MIROC-ES2L")
plot_npp(ax2, "~/Downloads/cmip", "ssp585", "MIROC-ES2L", "r1i1p1f2", "gn", "201501-210012", "x","y", linestyle="--", label="MIROC-ES2L")

# Plot CMIP6, ISPL-CM6A-LR
plot_npp(ax1, "~/Downloads/cmip", "ssp126", "IPSL-CM6A-LR", "r1i1p1f1", "gn", "201501-210012", "x","y", 60, True, "area", linestyle="-", label="IPSL-CM6A-LR")
plot_npp(ax2, "~/Downloads/cmip", "ssp585", "IPSL-CM6A-LR", "r1i1p1f1", "gn", "201501-210012", "x","y", 60, True, "area", linestyle="--", label="IPSL-CM6A-LR")

# Plot CMIP6, ACCESS-ESM1-5
plot_npp(ax1, "~/Downloads/cmip", "ssp126", "ACCESS-ESM1-5", "r10i1p1f1", "gn", "201501-210012", "i","j",linestyle="-", label="ACCESS-ESM1-5")
plot_npp(ax2, "~/Downloads/cmip", "ssp585", "ACCESS-ESM1-5", "r10i1p1f1", "gn", "201501-210012", "i","j",linestyle="--",  label="ACCESS-ESM1-5")

# MPI, RCP2.6/RCP8.5
plot_npp(ax1, "~/Downloads/cmip", "rcp26", "MPI-ESM-LR", "r1i1p1", None, "200601-210012", "i","j", linestyle="-", label="MPI-ESM-LR")
plot_npp(ax2, "~/Downloads/cmip", "rcp85", "MPI-ESM-LR", "r1i1p1", None, "200601-210012", "i","j", linestyle="--", label="MPI-ESM-LR")

# IPSL, RCP2.6/RCP8.5
plot_npp(ax1, "~/Downloads/cmip", "rcp26", "IPSL-CM5A-MR", "r1i1p1", None, "200601-210012", "i","j", linestyle="-", label="IPSL-CM5A-MR")
plot_npp(ax2, "~/Downloads/cmip", "rcp85", "IPSL-CM5A-MR", "r1i1p1", None, "200601-210012", "i","j", linestyle="--", label="IPSL-CM5A-MR")

No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
/Users/yingrui/miniforge3/envs/workspace/lib/python3.10/site-packages/xarray/conventions.py:523: SerializationWarning: variable 'intpp' has multiple fill values {1e+20, 1e+20}, decoding all values to NaN.
  new_vars[k] = decode_cf_variable(
/Users/yingrui/miniforge3/envs/workspace/lib/python3.10/site-packages/xarray/conventions.py:523: SerializationWarning: variable 'areacello' has multiple fill values {1e+20, 1e+20}, decoding all values to NaN.
  new_vars[k] = decode_cf_variable(
/var/folders/rq/vks10_qx2l9b08d_pygvwfzh0000gn/T/ipykernel_56209/2353424678.py:43: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex

In [6]:
from cgeniepy.foram import ForamModel as fm
from cgeniepy.grid import GENIE_grid_vol
import numpy as np
import pandas as pd

mod1p5 = fm("../model/worjh2.RpCO2_Rp13CO2.Albani.2100.1p5deg")
mod4 = fm("../model/worjh2.RpCO2_Rp13CO2.Albani.2100.4deg")

time_label = mod1p5.select_var("time").array.values - 4.5
time_label = pd.to_datetime(time_label, format='%Y')

vol = GENIE_grid_vol() * 1E9
d_to_y = 365
mmol_to_g = 12 * 1E-3
g_to_Pg = 1E-15

d1p5, d4 = [], []
for i in range(len(time_label)):
    npp_rate_1p5 = mod1p5.select_var("eco2D_Uptake_Fluxes_C").isel(time=i).array
    npp_total_1p5 = np.nansum(npp_rate_1p5 * vol * d_to_y * mmol_to_g * g_to_Pg)

    npp_rate_4 = mod4.select_var("eco2D_Uptake_Fluxes_C").isel(time=i).array
    npp_total_4 = np.nansum(npp_rate_4 * vol * d_to_y * mmol_to_g * g_to_Pg)

    d1p5.append(npp_total_1p5)
    d4.append(npp_total_4)

d1p5 = np.array(d1p5)
d1p5 = (d1p5/d1p5[0] - 1) * 100

d4= np.array(d4)
d4 = (d4/d4[0] - 1) * 100

ax1.plot(time_label, d1p5, color="k", label="this study")
ax2.plot(time_label, d4, color="k", linestyle="--", label="this study")

legend = ax2.legend(loc='center left', bbox_to_anchor=(1.05, 0.5))
ax1.set_title("Low emission")
ax2.set_title("High emission")

ax1.set_ylabel("NPP change (%)")
fig.savefig("../output/NPP_vs_CMIP.png", bbox_inches='tight', dpi=400)

/Users/yingrui/miniforge3/envs/workspace/lib/python3.10/site-packages/xarray/core/variable.py:314: UnitStrippedWarning: The unit of the quantity is stripped when downcasting to ndarray.
  data = np.asarray(data)
